# Package Install

In [ ]:
conda install numpy
conda install -c conda-forge matplotlib
conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia

# Import Package

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
%matplotlib inline

# Condition

In [ ]:
seed=0
TEMP=269
np.random.seed(seed )
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Data Loader

In [ ]:
class DataLoader(object):
    
    def __init__(self, choiced_particle_num=3):
        h_ice_ox_data = np.load('./ice/{}K_ice_data/1h_ice_oxygen.npy'.format(TEMP))
        h_ice_h1_data= np.load('./ice/{}K_ice_data/1h_ice_hydrogen1.npy'.format(TEMP))
        h_ice_h2_data= np.load('./ice/{}K_ice_data/1h_ice_hydrogen2.npy'.format(TEMP))
        h_ice_box = np.load('./ice/{}K_ice_data/1h_ice_box.npy'.format(TEMP))
        c_ice_ox_data = np.load('./ice/{}K_ice_data/1c_ice_oxygen.npy'.format(TEMP))
        c_ice_h1_data= np.load('./ice/{}K_ice_data/1c_ice_hydrogen1.npy'.format(TEMP))
        c_ice_h2_data= np.load('./ice/{}K_ice_data/1c_ice_hydrogen2.npy'.format(TEMP))
        c_ice_box = np.load('./ice/{}K_ice_data/1c_ice_box.npy'.format(TEMP))
        wat_ox_data = np.load('./wat/{}K_wat_data/water_oxygen.npy'.format(TEMP))
        wat_h1_data = np.load('./wat/{}K_wat_data/water_hydrogen1.npy'.format(TEMP))
        wat_h2_data = np.load('./wat/{}K_wat_data/water_hydrogen2.npy'.format(TEMP))
        wat_box = np.load('./wat/{}K_wat_data/water_box.npy'.format(TEMP))
        
        h_ice_data=np.stack([h_ice_h1_data,h_ice_h2_data,h_ice_ox_data],2)
        c_ice_data=np.stack([c_ice_h1_data,c_ice_h2_data,c_ice_ox_data],2)
        wat_data=np.stack([wat_h1_data,wat_h2_data,wat_ox_data],2)
        
        h_ice_label=np.zeros_like(h_ice_data[:,0,0,0])
        c_ice_label=np.ones_like(c_ice_data[:,0,0,0])
        wat_label=np.full_like(wat_data[:,0,0,0],2)
        
        self.choiced_particle_num = choiced_particle_num#選択粒子数
        self.data_size = h_ice_data.shape[0] +c_ice_data.shape[0]+wat_data.shape[0]
        self.maxlength=1.5*0.31668
        
        #print(h_ice_data.shape)
        #print(c_ice_data.shape)
        
        x=[]
        for i in range(h_ice_data.shape[0]):
            x.append(h_ice_data[i])#[スナップショット，粒子，座標]
        for i in range(c_ice_data.shape[0]):
            x.append(c_ice_data[i])
        for i in range(wat_data.shape[0]):
            x.append(wat_data[i])
            
        label = np.concatenate([h_ice_label, c_ice_label,wat_label],0)
        box = np.concatenate([h_ice_box, c_ice_box,wat_box],0)
        
        
        self.x_train, self.x_test, self.label_train, self.label_test, self.box_train, self.box_test = self.train_test_split(x, label, box, test_size=0.2)
        self.x_train, self.x_val, self.label_train, self.label_val, self.box_train, self.box_val = self.train_test_split(self.x_train, self.label_train, self.box_train, test_size=0.2)
        
        print(len(self.x_train),self.x_train[0].shape)
        print(len(self.x_test),self.x_test[0].shape)
        print(len(self.x_val),self.x_val[0].shape)
    def train_test_split(self,x,label,box,test_size=0.2):
        data_size=len(x)
        test_batch=int(len(x)*test_size)
        test_index=np.sort(np.random.choice(data_size,test_batch,replace=False))
        train_index=np.delete(np.array([i for i in range(len(x))]),test_index)
        x_test=[x[i].copy() for i in test_index]
        x_train=[x[i].copy() for i in train_index]
        label_test=label[test_index]
        label_train=label[train_index]
        box_test=box[test_index]
        box_train=box[train_index]
        return x_train,x_test,label_train,label_test,box_train,box_test
    
    
    def get_train_batch(self, batch_size):
        return self._get_batch(batch_size, self.x_train, self.label_train, self.box_train)
    
    
    def get_test_batch(self, batch_size):
        return self._get_batch(batch_size, self.x_test, self.label_test, self.box_test)
    
    
    def get_val_batch(self, batch_size):
        return self._get_batch(batch_size, self.x_val, self.label_val, self.box_val)
    
    
    def _get_batch(self, batch_size, x, label, box):
        batch_idx = np.random.randint(0,len(x), batch_size)#batch_size個のランダムな配列番号
        batch_x = [x[i].copy() for i  in batch_idx]
        batch_label, batch_box = label[batch_idx], box[batch_idx]
        batch_x, batch_label = self._choice_particle(batch_x.copy(), batch_label, batch_box)
        return batch_x, batch_label
    
    
    def _choice_particle(self, x, label, box):
        x=x.copy()
        choiced_x = np.zeros((len(x), self.choiced_particle_num, x[0].shape[1], x[0].shape[2]))
        for i in range(len(x)):#各スナップショット分繰り返す
            center_particle_idx = np.random.randint(0, x[i].shape[0])
            dx = x[i][ :, 2, 0:3] - x[i][center_particle_idx, 2, 0:3]
            for coordinate in range(3):#center_particleを中心にする
                l1 = np.where(dx[:,  coordinate] > box[i, coordinate]/2, -box[i, coordinate], 0)
                l2 = np.where(dx[:,  coordinate] < -box[i, coordinate]/2, +box[i, coordinate], 0)
                x[i][ :, 0, coordinate] = x[i][ :, 0, coordinate] + l1 + l2
                x[i][ :, 1, coordinate] = x[i][ :, 1, coordinate] + l1 + l2
                x[i][ :, 2, coordinate] = x[i][ :, 2, coordinate] + l1 + l2
            d = np.linalg.norm(x[i][ :, 2,0:3] - x[i][center_particle_idx, 2, 0:3], axis = 1)
            arg = np.argsort(d)[:self.choiced_particle_num]#距離の近い順に並べてchoiced_particle_num個持ってくる
            choiced_x[i, :, :, :] = x[i][arg, :, :].copy()
            reshaped_x=np.concatenate([choiced_x[:,:,0,:],choiced_x[:,:,1,:],choiced_x[:,:,2,:]],1)
        return reshaped_x, label

# Model

### We used TeaNet as our model; the implementation of TeaNet can be found in the paper [1].The input to TeaNet is three-dimensional coordinate data, the form is (batch size, number of particles, 3), and the output is a predicted scalar value (batch size, 1).

[1]Takamoto, S.; Izumi, S.; Li, J. TeaNet: Universal neural network interatomic potential
inspired by iterative electronic relaxations. Comput. Mater. Sci. 2022, 207, 111280. https://doi.org/10.1016/j.commatsci.2022.111280

In [ ]:
class TeaNet(nn.Module):
    def __init__(self,batch_size,molecule_num,particle_num):
        super(TeaNet,self).__init__()
        self.batch_size=batch_size
        self.molecule_num=molecule_num
        self.particle_num=particle_num
        
    def forward(self,x):
        return y

# Learning Program

In [ ]:
def main(batch_size, molecule_num):   
    
    if torch.cuda.is_available():
        print("GPU",torch.cuda.is_available())
        device=torch.device("cuda")
        print(device)
    
    torch.autograd.set_detect_anomaly(True)
    
    iters_num = 400000 + 5
    delta = 1e-7
    
    dataloader = DataLoader(choiced_particle_num=molecule_num)
    
    x_val, label_val = dataloader.get_val_batch(batch_size)
    x_val = torch.from_numpy(x_val).to(torch.float32).to(device)
    label_val = torch.from_numpy(label_val).to(torch.long).to(device)
    
    particle_num=x_val.shape[1]
    
    net = TeaNet(batch_size,molecule_num,particle_num).to(device) 
    #print(net)
    #print(list(net.parameters()))
    loss_list_train = []
    accuracy_list_train = []       
    loss_list_val = []
    accuracy_list_val = []
    
    learning_rate = 1e-3
    optimizer = torch.optim.RMSprop(net.parameters(), lr=learning_rate)
    scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=int(100000),gamma=0.5)
    loss=nn.CrossEntropyLoss()
    softmax=nn.Softmax(dim=1)
    
    for itr in range(iters_num+1):
        x_batch, label_batch = dataloader.get_train_batch(batch_size)
        x_batch = torch.from_numpy(x_batch).to(torch.float32).to(device)
        label_batch = torch.from_numpy(label_batch).to(torch.long).to(device)
        
        
        net.train()
        y_pred=net(x_batch)
        loss_value_train=loss(y_pred,label_batch)
        
        with torch.no_grad():
            y_pred=softmax(y_pred)
            _,y_pred=torch.max(y_pred ,1)
            correct_prediction_train = torch.eq(y_pred, label_batch).to(torch.float32)
            accuracy_value_train = torch.mean(correct_prediction_train.to(torch.float32))
            
            loss_list_train.append(loss_value_train.detach().cpu().numpy().copy())
            accuracy_list_train.append(accuracy_value_train.detach().cpu().numpy().copy())
            print('Loss {}: {}'.format(itr+1,loss_value_train))
            print('Accuracy{} : {}'.format(itr+1,accuracy_value_train))
            
            optimizer.zero_grad()
            loss_value_train.backward()
            optimizer.step()
            scheduler.step()
            
            if itr % 100 == 0 and itr != 0:
                net.eval()
                y_val=net(x_val)
                loss_value_val=loss(y_val,label_val)
                
                y_val=softmax(y_val)
                _,y_val=torch.max(y_val ,1)
                correct_prediction_val = torch.eq(y_val, label_val).to(torch.float32)
                accuracy_value_val=torch.mean(correct_prediction_val)
                loss_list_val.append(loss_value_val.detach().cpu().numpy().copy())
                accuracy_list_val.append(accuracy_value_val.detach().cpu().numpy().copy())
                print('TestAccuracy{} : {}'.format(itr+1,accuracy_value_val))

# Run

In [ ]:
batch_size =512
for particle_num in [3,5,7,9]:
    print(particle_num)
    main(batch_size,particle_num)
    clear_output()